In [1]:
import random
import math
import cv2
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass, field

In [2]:
from ipynb.fs.full.wspShannonEvaluation import wspShannonEvaluation

In [3]:
@dataclass
class Elephant:
    chrom: field(default_factory=lambda: [])
    cost: float = 0

In [4]:
def sort_elephants(elephants, fitness, n, H):
    for i in range(n): # 
        fitness[i] = -wspShannonEvaluation(H, elephants[i])
    
    indice = np.argsort(fitness)
    fitness.sort()

    # Ranking the elephants
    rank = [0]*n
    for i in range(n):
        rank[i] = elephants[indice[i]]

    elephants = rank

    return elephants, fitness

In [5]:
def CalculateClanCenter(Clan, cindex, dim, numElephantInEachClan):
    ClanCenter = np.zeros(dim)

    for Elephantindex in range(numElephantInEachClan):
        ClanCenter = ClanCenter + Clan[cindex][Elephantindex].chrom

    ClanCenter = (1/numElephantInEachClan)*ClanCenter
    return ClanCenter
    

In [6]:
def ClearDups(NewClan, Lb, Ub):
    """Clear Duplicates"""
    if len(Lb) == 1:
        for i in range(len(NewClan)):
            Chrom1 = np.sort(NewClan[i].chrom)
            for j in range(i+1, len(NewClan)):
                Chrom2 = np.sort(NewClan[j].chrom)
                if np.array_equal(Chrom1, Chrom2):
                    parnum = round(len(NewClan[j].chrom) * np.random.uniform(0, 1))
                    NewClan[j].chrom[parnum] = math.floor(Lb + (Ub - Lb + 1) * np.random.uniform(0, 1))
    else:
        for i in range(len(NewClan)):
            Chrom1 = np.sort(NewClan[i].chrom)
            for j in range(i+1, len(NewClan)):
                Chrom2 = np.sort(NewClan[j].chrom)
                if np.array_equal(Chrom1, Chrom2):
                    parnum = round(len(NewClan[j].chrom) * np.random.uniform(0, 1))
                    NewClan[j].chrom[parnum] = math.floor(Lb[parnum] + (Ub[parnum] - Lb[parnum] + 1) * np.random.uniform(0, 1))

    return NewClan

In [7]:
def FeasibleFunction(NewClan, n, dim, Lb, Ub):
    for i in range(n):
        for j in range(dim):
            NewClan[i].chrom[j] = np.clip(NewClan[i].chrom[j], Lb[j], Ub[j])
            NewClan[i].chrom.sort()

    return NewClan

In [8]:
def CostFunction(NewClan,H, n, dim, Lb, Ub):
    for i in range(len(NewClan)):
        threshs = NewClan[i].chrom
        NewClan[i].cost = -wspShannonEvaluation(H, threshs.astype(int))
    
    return NewClan

In [9]:
def PopSort(NewClan):
    popsize = len(NewClan)
    Cost = np.zeros(popsize)

    for i in range(popsize):
        Cost[i] = NewClan[i].cost
    
    indice = np.argsort(Cost)
    Cost.sort()

    # Ranking the elephants
    rank = [0]*popsize
    for i in range(popsize):
        rank[i] = NewClan[indice[i]].chrom

    for i in range(popsize):
        NewClan[i].chrom = rank[i]
        NewClan[i].cost = Cost[i]

    return NewClan
    

In [10]:
def CombineClan(NewClan, n, nClan, dim):
    j = 0
    popindex = 0
    Population = [Elephant(np.zeros(dim)) for w in range(n)]

    while popindex < n:
        for clanindex in range(nClan):
            Population[popindex] = NewClan[clanindex][j]
            popindex += 1

        j += 1
    
    return Population

In [11]:
def wspElephantHerding(n, dim, nkE, nClan, alpha, beta, maxGeneration, H):
    """"
    Elephant Herding Algorithm

    :param n: number of elephants
    :param d: dimension
    :param nkE: (elitism parameter) how many of the best elephants to keep from one generation to the next
    :param nClan: Number of clans
    :param alpha: Impact facotr of matriarch
    :param beta: attractivity factor
    :param maxGeneration: number of max generation
    :param H: histogram

    :return: thresholding set fi = {l1, l2, ..., ld }
    """
    t = 0
    keep = nkE
    Lb = 1*np.zeros(dim, dtype=int)
    Ub = 255*np.ones(dim, dtype=int)
    numElephantInEachClan = round(n/nClan)
    nEvaluations = n
    random.seed(0) # Reset the random generator
    np.random.seed(0)

    elephants = []

    for _ in range(n): # generate elephants with d-dimensional solution
        elephant = random.sample(range(1,255), dim)
        elephant.sort()
        elephants.append(elephant)

    fitness = np.zeros(n) # Initial fitness of each elephant

    chromKeep = np.zeros((keep, dim), dtype=int)
    costKeep = np.zeros(keep)
    Clan = [[Elephant(np.zeros(dim)) for x in range(numElephantInEachClan)] for y in range(nClan)]

    while t < maxGeneration:
        # Sort all the elephants according to their fitness.
        elephants, fitness = sort_elephants(elephants, fitness, n, H)

        """           Elitism Strategy           """
        # Save the best elephants in a temporary array.
        for i in range(keep):
            chromKeep[i] = elephants[i]
            costKeep[i] = fitness[i]

        """Divide the whole elephant population into some clans according to their fitness."""
        j = 0
        popIndex = 0

        while popIndex < n:
            for cindex in range(nClan):
                Clan[cindex][j] = Elephant(elephants[popIndex], fitness[popIndex])
                popIndex += 1

            j += 1

        """              Clan Updating Operator              """
        j = 0
        popIndex = 0
        NewClan = [[Elephant(np.zeros(dim)) for x in range(numElephantInEachClan)] for y in range(nClan)]
        
        while popIndex < n:
            for cindex in range(nClan):
                ClanCenter = CalculateClanCenter(Clan, cindex, dim, numElephantInEachClan)
                NewClan[cindex][j].chrom = Clan[cindex][j].chrom + alpha * (np.array(Clan[cindex][0].chrom) - np.array(Clan[cindex][j].chrom)) * np.random.uniform(0, 1, dim)

                if np.sum(NewClan[cindex][j].chrom - Clan[cindex][j].chrom) == 0:
                    NewClan[cindex][j].chrom = beta * ClanCenter

                popIndex += 1
            
            j+=1
        
        """           Separating Operator          """
        for cindex in range(nClan):
            NewClan[cindex][len(NewClan[cindex])-1].chrom = Lb + (Ub - Lb + 1)*np.random.uniform(0, 1, dim)
        
        """           Evaluate NewClan          """
        SavePopSize = n
        for i in range(nClan):
            n = numElephantInEachClan
            # Make sure the population does not have duplicates.
            NewClan[i] = ClearDups(NewClan[i], Lb, Ub)
            # Make sure each individual is legal.
            NewClan[i] = FeasibleFunction(NewClan[i], n , dim, Lb, Ub)
            # Calculate cost
            NewClan[i] = CostFunction(NewClan[i],H, n, dim, Lb, Ub)
            # the number of fitness evaluations
            nEvaluations = nEvaluations +  n
            # Sort from best to worst
            NewClan[i] = PopSort(NewClan[i])
        n = SavePopSize

        """Combine two subpopulations into one and rank monarch butterflis"""
        Population = CombineClan(NewClan, n, nClan, dim)
        Population = PopSort(Population)
        
        """           Elitism Strategy          """
        # Replace the worst with the previous generation's elites.
        p_size = len(Population)-1
        for k in range(keep):
            Population[p_size-k].chrom = chromKeep[k]
            Population[p_size-k].cost = costKeep[k]

        # Sort from best to worst
        Population = PopSort(Population)
        
        t+=1
    return (Population[0].chrom).astype(int)
        

In [12]:
n = 10
nkE = 2
nClan = 5
alpha = 0.5
beta = 0.1
d = 3
maxGeneration = 100

image = cv2.imread("images/lenna.png")
img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hist, _ = np.histogram(img.ravel(),256,[0,255])
hist = hist/np.sum(hist)

best_thresholds = wspElephantHerding(n, d, nkE, nClan, alpha, beta, maxGeneration, hist)


In [13]:
elephants = [ [45,   101,   167],
        [72,   140,   244],
        [37,   108,   233],
        [124,   204,   244],
        [26,   161,   233],
        [167,   202,   244],
        [34,   208,   231],
        [173,   190,   193],
        [11,   216,   238],
        [41,   246,   247]
    ]

fitness = [-16.0252474129072,
-15.0904785421023,
-15.086017632213,
-14.8607819769433,
-14.7310525174928,
-14.4548588138481,
-14.347506441319,
-12.8798370250284,
-12.6226512328904,
-10.7101810045556]

dim = 3
numElephantInEachClan = 2
Clan = [[Elephant(np.zeros(dim))] * numElephantInEachClan for y in range(nClan)]

"""Divide the whole elephant population into some clans according to their fitness."""
j = 0
popIndex = 0

while popIndex < n:
    for cindex in range(nClan):
        Clan[cindex][j] = Elephant(elephants[popIndex], fitness[popIndex])
        popIndex += 1

    j += 1

j = 0
popIndex = 0
NewClan = [[Elephant(np.zeros(dim))] * numElephantInEachClan for y in range(nClan)]

while popIndex < n:
    for cindex in range(nClan):
        ClanCenter = CalculateClanCenter(Clan, cindex, dim, numElephantInEachClan)
        NewClan[cindex][j].chrom = Clan[cindex][j].chrom + alpha * (np.array(Clan[cindex][0].chrom) - np.array(Clan[cindex][j].chrom)) * np.random.uniform(0, 1, dim)

        if np.sum(NewClan[cindex][j].chrom - Clan[cindex][j].chrom) == 0:
            NewClan[cindex][j].chrom = beta * ClanCenter

        popIndex += 1
    
    j+=1